In [2]:
import os

os.chdir("/work")

# %% 準備
import pandas as pd
from utils.logger import app_logger
from utils.config_loader import get_template_config
from logic.manage.utils.csv_loader import load_all_filtered_dataframes
from logic.manage.utils.load_template import load_master_and_template
from IPython.display import display
import re
from logic.manage.factory_report import process
from logic.manage.utils.excel_tools import create_label_rows_generic, sort_by_cell_row

In [3]:
# 表示ラベルマップ（処理対象名として使う）
csv_label_map = {"yard": "ヤード一覧", "shipping": "出荷一覧", "receive": "受入一覧"}

debug_receive = "/work/data/output/debug_receive.parquet"
debug_shipping = "/work/data/output/debug_shipping.parquet"
debug_yard = "/work/data/output/debug_yard.parquet"

dfs = {
    "receive": pd.read_parquet(debug_receive),
    "shipping": pd.read_parquet(debug_shipping),
    "yard": pd.read_parquet(debug_yard),
}  # テスト用CSV
# dfs
df_shipping = dfs["shipping"]
df_yard = dfs["yard"]

In [ ]:
def process_factory_report(dfs):
    from logic.manage.factory_report import process

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["yard"]
    master_csv = load_master_and_template(master_path)

    # 工場日報からdfを読込
    df_factory = process(dfs)

    # 必要な箇所を抜出
    after_master_csv = calculate(master_csv, df_factory)

    display(df_factory)


process_factory_report(dfs)

2025-04-28 13:22:09,974 [INFO] (factory_report.py:27) [fb6959af9847/root] [テンプレート設定読込] key=factory_report, files=['yard', 'shipping']
2025-04-28 13:22:10,010 [INFO] (factory_report.py:35) [fb6959af9847/root] ▶️ 出荷処分データ処理開始
/work/utils/data_cleaning.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['8339' '5238' '8340' '8085' '6421' '8337' '8215' '8336' '8327' '8327'
 '8327' '8327' '705' '1185' '8076' '8077' '5199' '8334' '6797' '8338'
 '8341' '8080' '349' '6786' '5227' '8329' '8360' '8363' '8398' '8679'
 '339' '8082']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[valid, col] = df.loc[valid, col].apply(lambda x: str(int(float(x))))
2025-04-28 13:22:10,092 [INFO] (factory_report_shobun.py:40) [fb6959af9847/root] ✅ 出荷処分の帳票生成が完了しました。
2025-04-28 13:22:10,094 [INFO] (factory_report.py:38) [fb6959af9847/root] ▶️ 出荷有価データ処理開始
2025-04-28 13:22:10,137 [INFO] (su

,大項目,セル,値,カテゴリ
0,月日,O1,3月31日,NaN
1,和暦,M1,令和7年,NaN
2,,F19,紙、ダンボール,有価
3,,I19,ペットボトル,有価
4,,C19,スクラップ,有価
...,...,...,...,...
112,ヤード（金庫）,F50,180.0,ヤード
113,ヤード（シュレッダー）,C50,11520.0,ヤード
114,総合計,O50,101050.0,NaN
115,合計_ヤード,未設定,13360.0,ヤード


In [ ]:
from utils.value_setter import set_value_fast
def calculate(master_csv, df_factory):

    df_factory["大項目"]==

    match_columns = ["大項目"]
    match_value = ["和暦"]
    set_value_fast(master_csv, match_columns, match_value, to_japanese_era(today))